**jumbo rolls are folded in**

**changeover between jumbos are folded in**

need to add 'pulled from inventory' rows to the schedule.

or maybe show when an order in the future has been fullfilled.

then optimize for knife changes

# Simple Genetic Algorithm to Optimize Deckle Loss

In [70]:
import sys
sys.path.append("../")

In [71]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy
from utils import *
from genetic import *
from IPython.display import clear_output
import time
import datetime
import re

In [72]:
df = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')

# the input schedule appears as a data table on the dashboard
# this is used to select what to deckle
df_input_schedule = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')
df_input_schedule.insert(1, 'Technology', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'tech')))
df_input_schedule.insert(2, 'Color', df_input_schedule['Description'].apply(lambda x: x.split(';')[1] if type(x) == str else None))
df_input_schedule.insert(3, 'Width', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'width')))
df_input_schedule = df_input_schedule[[col for col in df_input_schedule.columns if 'Unnamed' not in str(col)]]
temp = df_input_schedule[['Customer Name', 'Technology', 'Color', 'Width', 'CYCLE / BUCKET',
                          'Description', 'Total LM Order QTY',
                          'Scheduled Ship Date', 'Date order is complete']]

In [73]:
# temp = temp[temp['Scheduled Ship Date'].apply(lambda x: type(x) == int)]                          
temp_json = temp.to_json()

In [74]:
temp2 = pd.read_json(temp_json, 
                     convert_dates=['Scheduled Ship Date', 'Date order is complete'])

## to_datetime and to_json use default units ms and ns, respectively
temp2['Scheduled Ship Date'] = pd.to_datetime(temp2['Scheduled Ship Date'], 
                                              errors='ignore', unit='ms')

In [75]:
### not currently used, grabs indicated information
customer_names = [i for i in df['Customer Name'].unique() if type(i) == str]
technologies = list(set([parse_description(i, 'tech') for i in df['Description'] 
                         if (type(i) == str)]))
technologies = [i for i in technologies if i]
colors = list(set([i[1] for i in list(df['Description']
                .str.split(';')) if type(i) == list]))
cycles = list(df['CYCLE / BUCKET'].unique())

In [76]:
customer = 'P & G'
technology = 'SAM' # Technology -> 
color = 'WHITE'
cycle = 'CYCLE 2'

df_filtered = df.loc[df['Customer Name'] == customer]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(technology)]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(color)] #CYAN, TEAL
df_filtered = df_filtered.loc[df_filtered['CYCLE / BUCKET'] == cycle]
df_filtered.insert(0, 'Block', 1)
df_filtered = df_filtered.reset_index(drop=True)

In [77]:
### Detects if there is a break in the rolls coming of the
### line

prev_index = df_filtered.index[0] - 1
block = 1
for index in df_filtered.index:
    if index != prev_index + 1:
        block += 1
    df_filtered.loc[index, 'Block'] = block
    prev_index = index

In [78]:
df_filtered['Width'] = pd.DataFrame(list(pd.DataFrame(list(df_filtered['Description']
                                                  .str.split(';')))[0].str.split('UN0')))[1]
lm = list(df_filtered.groupby('Description')['Total LM Order QTY'].sum().values)
widths = list(df_filtered.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
doffs_in_jumbo = 6
B = 4160
L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
w = list(np.array(widths) + np.array(neckin)) # the values used in the actual computation
q = [math.ceil(x/L) for x in lm] # jumbos needed per width, rounded up
s = BinPackingExample(w, q) # material orders (list of widths, 1 width = 1 jumbo)

print('The important variables', end='\n\n')
print('widths: {} (mm)'.format(widths))
print('neck in: {} (mm)'.format(neckin))
print('L (m): {}'.format(L))
print('undeckled jumbos needed: {}'.format(q))

The important variables

widths: [154, 158, 170, 208, 218, 234] (mm)
neck in: [4, 4, 5, 7, 7, 7] (mm)
L (m): 102000
undeckled jumbos needed: [96, 32, 318, 95, 158, 461]


In [79]:
# customer = 'P & G'
# product = 'SAM' # Technology -> 
# color = 'WHITE'
# cycle = 'CYCLE 2'

# df = pd.read_excel('../data/berry/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
#                    sheet_name='Schedule')
# df = df.loc[df['Customer Name'] == customer]
# df = df.loc[df['Description'].str.contains(product)]
# df = df.loc[df['Description'].str.contains(color)] #CYAN, TEAL
# df = df.loc[df['CYCLE / BUCKET'] == cycle]
# df = df.reset_index(drop=True)
# df['Width'] = pd.DataFrame(list(pd.DataFrame(list(df['Description']
#                                                   .str.split(';')))[0].str.split('UN0')))[1]
# lm = list(df.groupby('Description')['Total LM Order QTY'].sum().values)
# widths = list(df.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
# doffs_in_jumbo = 6
# B = 4160
# L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

# neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
# w = list(np.array(widths) + np.array(neckin))
# q = [math.ceil(x/L) for x in lm]
# print("max doffs needed without stock cutting: {}".format(q))
# s = BinPackingExample(w, q)
# df_json = df.to_json()

if we gave each order its own entire row we would produce a lot of wasted material

# Run Deckle Optimizer

In [80]:
sol, loss = find_optimum(s, B, widths, neckin, max_unique_products=4, loss_target=2)

gene time: 1.6, chromosome time: 1599046989.4, ffd time: 1.6
loss: 0.962, best loss: 0.962, steps: 8, gene count: 5, unique doffs: 10, max unique on doff: 4
                          Formula Doffs Loss
0           170x9 + 218x5 + 234x6    31   14
1                          234x17    16   63
2                          208x19     4   75
3                          154x26     2   52
4                          154x25     1  210
5                  154x19 + 158x7     1   24
6                          158x25     1  110
7                          170x23     1  135
8                  170x15 + 208x7     1   30
9  170x1 + 208x12 + 218x3 + 234x3     1    7


# Build Layout for Dashboard View

In [81]:
layout_summary(sol, widths, neckin, B)

0      1      2      3      4      5      6      7      8      9  \
Doffs                                                                         
1      154.0  154.0  154.0  154.0  154.0  154.0  154.0  154.0  154.0  154.0   
2      154.0  154.0  154.0  154.0  154.0  154.0  154.0  154.0  154.0  154.0   
1      158.0  158.0  158.0  158.0  158.0  158.0  158.0  154.0  154.0  154.0   
1      158.0  158.0  158.0  158.0  158.0  158.0  158.0  158.0  158.0  158.0   
1      170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0  170.0   
1      208.0  208.0  208.0  208.0  208.0  208.0  208.0  170.0  170.0  170.0   
4      208.0  208.0  208.0  208.0  208.0  208.0  208.0  208.0  208.0  208.0   
31     218.0  170.0  170.0  234.0  170.0  170.0  218.0  170.0  234.0  170.0   
1      234.0  234.0  234.0  218.0  218.0  218.0  208.0  208.0  208.0  208.0   
16     234.0  234.0  234.0  234.0  234.0  234.0  234.0  234.0  234.0  234.0   

       ...     18     19     20     21     22     23     24     25  Doffs  \
Doffs  ...                                                                  
1      ...  154.0  154.0  154.0  154.0  154.0  154.0  154.0    NaN      1   
2      ...  154.0  154.0  154.0  154.0  154.0  154.0  154.0  154.0      2   
1      ...  154.0  154.0  154.0  154.0  154.0  154.0  154.0  154.0      1   
1      ...  158.0  158.0  158.0  158.0  158.0  158.0  158.0    NaN      1   
1      ...  170.0  170.0  170.0  170.0  170.0    NaN    NaN    NaN      1   
1      ...  170.0  170.0  170.0  170.0    NaN    NaN    NaN    NaN      1   
4      ...  208.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN      4   
31     ...  234.0  218.0    NaN    NaN    NaN    NaN    NaN    NaN     31   
1      ...  170.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN      1   
16     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     16   

        Loss  
Doffs         
1      210.0  
2       52.0  
1       24.0  
1      110.0  
1      135.0  
1       30.0  
4       75.0  
31      14.0  
1        7.0  
16      63.0  

[10 rows x 28 columns]

# Run Master Schedule, Optimizing Late Orders

In [82]:
optimize_late_orders(sol, widths, neckin, df_filtered, L, DEBUG=False)

../utils.py:350: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Formula Doffs
0                           170x23     1
1                   170x15 + 208x7     1
2            170x9 + 218x5 + 234x6     3
3                           154x26     2
4            170x9 + 218x5 + 234x6     3
5            170x9 + 218x5 + 234x6     3
6            170x9 + 218x5 + 234x6     5
7                           234x17     1
8                           158x25     1
9                   154x19 + 158x7     1
10                          234x17    15
11           170x9 + 218x5 + 234x6    17
12  170x1 + 208x12 + 218x3 + 234x3     1
13                          154x25     1
14                          208x19     4

In [83]:
setup_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Deckle Parameters')
speed_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Product Parameters')

In [84]:
speed_df.loc[(speed_df['Description'].str.contains(technology)) &
             (speed_df['Description'].str.contains(color)), 'Slittter Speed (m/min)']

2    1300
3    1300
4    1300
5    1300
6    1300
7    1300
Name: Slittter Speed (m/min), dtype: int64

In [85]:
master = optimize_schedule(sol, widths, neckin, df_filtered, L, setup_df, speed_df, doffs_in_jumbo)

transition time: 0:15:00
run time: 1:18:27.692308
completion date/time: 2020-08-16 05:14:13.846172


In [86]:
master.to_csv('temp', date_format='%y-%m-%d %H:%M:%S')

In [87]:
def optimize_schedule(sol, widths, neckin, df_filtered, L, setup_df, speed_df, doffs_in_jumbo):
    
    #####################################################
    # Step 1: Create Schedule According to Order Sequence
    #####################################################
    extras = pd.DataFrame(np.zeros(len(widths))).T
    master2 = make_layout_registrar(sol, widths, neckin)
    extras.columns = master2.columns[:-1]
    schedule = []
    schedule_with_order_info = []
    layout_pattern = 0
    old_width = width = None
    completed_orders = []

    #### logic for jumbo rolls should go in first loop
    for row1 in df_filtered.index:
        clear_output(wait=True)
        ### select schedule data
        current_scheduled = df_filtered.iloc[row1][['Total LM Order QTY', 'Width', 'Scheduled Ship Date']]
        current_scheduled['Order Number'] = row1+1
        ship_date = current_scheduled['Scheduled Ship Date']
        doffs = math.ceil(current_scheduled['Total LM Order QTY'] / L) # QTY
        width = current_scheduled['Width']

        ### check if we've entered a new product
        ### no setup changes here, we need to see if the layout has changed
        if width == old_width:
            layout_pattern -= 1

        ### sort by the current scheduled width in master 2
        master2 = master2.sort_values(width, ascending=False).reset_index(drop=True)

        ### calc how many new doffs need to be made from inventory
        target_doffs = extras.iloc[0][width]
        print(pd.DataFrame(data=[[doffs, width, target_doffs]], 
                           columns=['required doffs', 'width', 'doffs made']))
        print(doffs, width, target_doffs)

        ### only proceed if doffs are available in layout registrar
        ### and not in inventory 
        if ((any(master2.loc[master2[width] > 0]['freq'] > 0)) 
            and (target_doffs <= doffs)): 

        ### go through the rows in the master2 registrar and check for
        ### layout patterns that contain our current width

        ### Every row is a new layout unless the outer 
        ### loop has incremented, then only maybe is it
        ### a new layout
            for row in master2.index: 

                # if none of the width are in the layout then break out
                if np.isnan(master2.iloc[row][width]): 
                    break
                layout_pattern += 1

                # looping through available doffs in the layout
                for count in range(master2.iloc[row]['freq'].astype(int)): 

                    # add to target doffs
                    target_doffs += master2.iloc[row][width] 

                    # add to number of times we've made this layout
                    master2.at[row, 'layout number'] = layout_pattern 


                    new_layout_and_scheduled_product = pd.concat([master2.iloc[row], 
                                                                  current_scheduled])
                    schedule_with_order_info.append(new_layout_and_scheduled_product)
                    schedule.append(master2.iloc[row])
                    master2.at[row, 'freq'] = master2.at[row, 'freq'] - 1

                    ### Tabluate the other widths that were made on the layout
                    for master_index in master2.iloc[row].index[:-2]:
                        if (math.isnan(master2.iloc[row][master_index]) == False):
                            ### tabulate extras for other widths in the layout
                            extras.iloc[0][master_index] = extras.iloc[0][master_index] +\
                                master2.iloc[row][master_index]
                            completed_orders.append
                    if target_doffs >= doffs:

                        ### add to registrar of completed orders.
                        completed_orders.append(current_scheduled['Order Number'])

                        ### now that we've finished this order, we want to see if we've completed any other orders as well


                        break
                ### for exiting nested for loop
                else:
                    continue
                break
        old_width = width
        extra = target_doffs - doffs
        extras.iloc[0][width] = extra

        print(extras)

    sorted_schedule_with_order_info = pd.DataFrame(schedule_with_order_info)
    sorted_schedule_with_order_info = sorted_schedule_with_order_info.reset_index(drop=True)
    sorted_schedule = pd.DataFrame(schedule)
    sorted_schedule = sorted_schedule.reset_index(drop=True)
    
    #####################################################
    # Step 2: Add Times Based on Rates/Changeovers
    #####################################################
    ### requires setup_df, speed_df
    
#     slitter_speed = speed_df.loc[(speed_df['Customer Name'] == customer) &
#              (speed_df['Description'].str.contains(technology)) &
#              (speed_df['Description'].str.contains(color)), 'Slittter Speed (m/min)'].reset_index(drop=True)[0]
    slitter_speed = 1300
    
    # add column 'completion date/time'
    # for row in schedule, calculate the completion time
    sorted_schedule_with_order_info['Completion Date'] = None

    ### choose starte date/time
    start_date_time = datetime.datetime(2020, 8, 12)
    print("start time: {}".format(start_date_time))

    ### set changeover times
    jumbo_change = setup_df.loc[setup_df['Slitter Set-up'] == 
                                'Jumbo roll only']['Time (minutes)'].values[0]
    jumbo_change = datetime.timedelta(minutes=jumbo_change)
    print("jumbo change: {}".format(jumbo_change))
    jumbo_and_knife_change = setup_df.loc[setup_df['Slitter Set-up'] == 
                                          'AMB & Arium']['Time (minutes)'].values[0]
    jumbo_and_knife_change = datetime.timedelta(minutes=jumbo_and_knife_change)
    print("jumbo and knife change: {}".format(jumbo_and_knife_change))

    ### print slitter speed and doff length
    print("slitter speed (m/min): {}".format(slitter_speed))
    print("doff length (m): {}".format(L))
    prior_completion_date_time = start_date_time
    match = '\d\d\d'
    layout_columns = [i for i in sorted_schedule_with_order_info.columns if re.match(match, i)]

    ### zero-out the 'prior layout' so algorithm knows jumbo + knife change
    prior_layout = sorted_schedule_with_order_info.iloc[0][layout_columns]
    for col in prior_layout.index:
        prior_layout[col] = 0
    layout_number = 0

    schedule_with_change_over = pd.DataFrame()
    ### transition times are for loading the current jumbo
    for row in sorted_schedule_with_order_info.index:

        layout = sorted_schedule_with_order_info.iloc[row][layout_columns]
        if all(prior_layout.fillna(0) == layout.fillna(0)):
            transition_time = jumbo_change
        else:
            transition_time = jumbo_and_knife_change
            layout_number += 1
        print("transition time: {}".format(transition_time))
        run_time = L / slitter_speed
        run_time = datetime.timedelta(minutes=run_time)
        print("run time: {}".format(run_time))
        completion_time = transition_time + run_time
        completion_date_time = prior_completion_date_time + completion_time
        print("completion date/time: {}".format(completion_date_time))

        ### change main df
        sorted_schedule_with_order_info['Completion Date'][row] = completion_date_time
        sorted_schedule_with_order_info['layout number'][row] = layout_number

        ### with changeover rows
        current_changeover = pd.DataFrame(sorted_schedule_with_order_info.columns)
        current_changeover = current_changeover.set_index(0)
        current_changeover = current_changeover.T
        current_changeover = current_changeover.append({'Order Number': 'Changeover'}, ignore_index=True)
        current_changeover['Completion Date'] = prior_completion_date_time + transition_time
        current_order = pd.DataFrame(sorted_schedule_with_order_info.iloc[row]).T
        current_combined = current_changeover.append(current_order, sort=False, ignore_index=True)

        schedule_with_change_over = pd.concat([schedule_with_change_over, current_combined])

        prior_completion_date_time = completion_date_time
        prior_layout = layout
        clear_output(wait=True)
        
    schedule_with_change_over = schedule_with_change_over.reset_index(drop=True)
    
    #####################################################
    # Step 3: Create Summary With Changeover Rows
    #####################################################

    master_schedule = pd.DataFrame()
    unformatted_schedule = pd.DataFrame()
    for index in schedule_with_change_over.index[1::2]: # pass through every jumbo
        order_number = schedule_with_change_over.iloc[index]['Order Number']

        ### these are columns specific to width layouts
        match = '\d\d\d'
        layout_columns = [i for i in schedule_with_change_over.columns if re.match(match, i)]


        deckle_layout =  deckle_order = pd.DataFrame(schedule_with_change_over.iloc[index]).T

        ### these are columns specific to order details
        order_columns = [i for i in deckle_order.columns if (i not in layout_columns)
                         & (i not in ['freq', 'layout number'])]

        ### for order we grab the last row since this is the real 'completion date'
        order = deckle_order[order_columns].reset_index(drop=True).iloc[-1].dropna()

        ### remove widths that are in the deckle opt. but not in the layout
        formula = deckle_layout[layout_columns].reset_index(drop=True).iloc[0].dropna()

        ### number of doffs for the given layout/order
        doffs = doffs_in_jumbo
        read_out = ''
        for i in range(formula.shape[0]-1): #for unique prods
            read_out = read_out + ("{}x{} + ".format(formula.index[i], 
                                                    formula.iloc[i].astype(int)))
        read_out = read_out + ("{}x{}".format(formula.index[-1], 
                                            formula.iloc[-1].astype(int))) #make last string w/o +
        current = pd.DataFrame([read_out, int(doffs), order]).T
        order = pd.DataFrame(order).T
        order = order.reset_index(drop=True)
        current = pd.DataFrame([read_out, int(doffs)]).T
        current.columns = ['Formula', 'Doffs']
        current = current.join(order)

        ### add changeover info
        current_changeover = pd.DataFrame(current.columns)
        current_changeover = current_changeover.set_index(0)
        current_changeover = current_changeover.T
        current_changeover = current_changeover.append({'Order Number': 'Changeover'}, ignore_index=True)
        current_changeover['Completion Date'] = schedule_with_change_over.iloc[index-1]['Completion Date']
        current_combined = current_changeover.append(current, sort=False, ignore_index=True)
        current_combined
        
        master_schedule = pd.concat([master_schedule, current_combined])
        master_schedule = master_schedule.reset_index(drop=True)
    return master_schedule

In [88]:
# run through schedule
# first roll
# change knife, jumbo, and manderal
# run at slitter speed
# look at next roll
# if same knife setup only change jumbo setup
# otherwise change jumbo + knife 
# loop

In [89]:
extras = pd.DataFrame(np.zeros(len(widths))).T
master2 = make_layout_registrar(sol, widths, neckin)
extras.columns = master2.columns[:-1]
schedule = []
schedule_with_order_info = []
layout_pattern = 0
old_width = width = None
completed_orders = []

#### logic for jumbo rolls should go in first loop
for row1 in df_filtered.index:
    clear_output(wait=True)
    ### select schedule data
    current_scheduled = df_filtered.iloc[row1][['Total LM Order QTY', 'Width', 'Scheduled Ship Date']]
    current_scheduled['Order Number'] = row1+1
    ship_date = current_scheduled['Scheduled Ship Date']
    doffs = math.ceil(current_scheduled['Total LM Order QTY'] / L) # QTY
    width = current_scheduled['Width']
    
    ### check if we've entered a new product
    ### no setup changes here, we need to see if the layout has changed
    if width == old_width:
        layout_pattern -= 1
        
    ### sort by the current scheduled width in master 2
    master2 = master2.sort_values(width, ascending=False).reset_index(drop=True)
    
    ### calc how many new doffs need to be made from inventory
    target_doffs = extras.iloc[0][width]
    print(pd.DataFrame(data=[[doffs, width, target_doffs]], 
                       columns=['required doffs', 'width', 'doffs made']))
    print(doffs, width, target_doffs)
    
    ### only proceed if doffs are available in layout registrar
    ### and not in inventory 
    if ((any(master2.loc[master2[width] > 0]['freq'] > 0)) 
        and (target_doffs <= doffs)): 
    
    ### go through the rows in the master2 registrar and check for
    ### layout patterns that contain our current width
    
    ### Every row is a new layout unless the outer 
    ### loop has incremented, then only maybe is it
    ### a new layout
        for row in master2.index: 
            
            # if none of the width are in the layout then break out
            if np.isnan(master2.iloc[row][width]): 
                break
            layout_pattern += 1
            
            # looping through available doffs in the layout
            for count in range(master2.iloc[row]['freq'].astype(int)): 
                
                # add to target doffs
                target_doffs += master2.iloc[row][width] 
                
                # add to number of times we've made this layout
                master2.at[row, 'layout number'] = layout_pattern 
                
              
                new_layout_and_scheduled_product = pd.concat([master2.iloc[row], 
                                                              current_scheduled])
                schedule_with_order_info.append(new_layout_and_scheduled_product)
                schedule.append(master2.iloc[row])
                master2.at[row, 'freq'] = master2.at[row, 'freq'] - 1

                ### Tabluate the other widths that were made on the layout
                for master_index in master2.iloc[row].index[:-2]:
                    if (math.isnan(master2.iloc[row][master_index]) == False):
                        ### tabulate extras for other widths in the layout
                        extras.iloc[0][master_index] = extras.iloc[0][master_index] +\
                            master2.iloc[row][master_index]
                        completed_orders.append
                if target_doffs >= doffs:
                    
                    ### add to registrar of completed orders.
                    completed_orders.append(current_scheduled['Order Number'])
                    
                    ### now that we've finished this order, we want to see if we've completed any other orders as well
                    
                    
                    break
            ### for exiting nested for loop
            else:
                continue
            break
    old_width = width
    extra = target_doffs - doffs
    extras.iloc[0][width] = extra
    
    print(extras)
    
sorted_schedule_with_order_info = pd.DataFrame(schedule_with_order_info)
sorted_schedule_with_order_info = sorted_schedule_with_order_info.reset_index(drop=True)
sorted_schedule = pd.DataFrame(schedule)
sorted_schedule = sorted_schedule.reset_index(drop=True)

   required doffs width  doffs made
0              21   234         3.0
21 234 3.0
   154  158  170  208  218  234
0  0.0  0.0 -1.0 -1.0 -3.0 -1.0


In [90]:
sorted_schedule_with_order_info

154   158   170   208  218   234  freq  layout number  \
0    NaN   NaN  23.0   NaN  NaN   NaN   1.0            1.0   
1    NaN   NaN  15.0   7.0  NaN   NaN   1.0            2.0   
2    NaN   NaN   9.0   NaN  5.0   6.0  31.0            4.0   
3    NaN   NaN   9.0   NaN  5.0   6.0  30.0            4.0   
4    NaN   NaN   9.0   NaN  5.0   6.0  29.0            4.0   
5   26.0   NaN   NaN   NaN  NaN   NaN   2.0            5.0   
6   26.0   NaN   NaN   NaN  NaN   NaN   1.0            5.0   
7    NaN   NaN   9.0   NaN  5.0   6.0  28.0            8.0   
8    NaN   NaN   9.0   NaN  5.0   6.0  27.0            8.0   
9    NaN   NaN   9.0   NaN  5.0   6.0  26.0            8.0   
10   NaN   NaN   9.0   NaN  5.0   6.0  25.0           10.0   
11   NaN   NaN   9.0   NaN  5.0   6.0  24.0           10.0   
12   NaN   NaN   9.0   NaN  5.0   6.0  23.0           10.0   
13   NaN   NaN   9.0   NaN  5.0   6.0  22.0           10.0   
14   NaN   NaN   9.0   NaN  5.0   6.0  21.0           10.0   
15   NaN   NaN   9.0   NaN  5.0   6.0  20.0           10.0   
16   NaN   NaN   9.0   NaN  5.0   6.0  19.0           10.0   
17   NaN   NaN   9.0   NaN  5.0   6.0  18.0           10.0   
18   NaN  25.0   NaN   NaN  NaN   NaN   1.0           11.0   
19  19.0   7.0   NaN   NaN  NaN   NaN   1.0           12.0   
20   NaN   NaN   NaN  19.0  NaN   NaN   4.0           13.0   
21   NaN   NaN   NaN  19.0  NaN   NaN   3.0           13.0   
22   NaN   NaN   NaN  19.0  NaN   NaN   2.0           13.0   
23   NaN   NaN   9.0   NaN  5.0   6.0  17.0           14.0   
24   NaN   NaN   9.0   NaN  5.0   6.0  16.0           14.0   
25   NaN   NaN   9.0   NaN  5.0   6.0  15.0           14.0   
26   NaN   NaN   9.0   NaN  5.0   6.0  14.0           14.0   
27   NaN   NaN   9.0   NaN  5.0   6.0  13.0           14.0   
28   NaN   NaN   9.0   NaN  5.0   6.0  12.0           13.0   
29   NaN   NaN   9.0   NaN  5.0   6.0  11.0           13.0   
30   NaN   NaN   9.0   NaN  5.0   6.0  10.0           13.0   
31   NaN   NaN   9.0   NaN  5.0   6.0   9.0           13.0   
32   NaN   NaN   9.0   NaN  5.0   6.0   8.0           13.0   
33   NaN   NaN   9.0   NaN  5.0   6.0   7.0           13.0   
34   NaN   NaN   9.0   NaN  5.0   6.0   6.0           13.0   
35   NaN   NaN   9.0   NaN  5.0   6.0   5.0           13.0   
36   NaN   NaN   9.0   NaN  5.0   6.0   4.0           13.0   
37   NaN   NaN   NaN   NaN  NaN  17.0  16.0           14.0   
38   NaN   NaN   NaN   NaN  NaN  17.0  15.0           14.0   
39   NaN   NaN   NaN   NaN  NaN  17.0  14.0           14.0   
40   NaN   NaN   NaN   NaN  NaN  17.0  13.0           14.0   
41   NaN   NaN   NaN   NaN  NaN  17.0  12.0           14.0   
42   NaN   NaN   NaN   NaN  NaN  17.0  11.0           14.0   
43   NaN   NaN   NaN   NaN  NaN  17.0  10.0           14.0   
44   NaN   NaN   NaN   NaN  NaN  17.0   9.0           14.0   
45   NaN   NaN   NaN   NaN  NaN  17.0   8.0           14.0   
46   NaN   NaN   NaN   NaN  NaN  17.0   7.0           14.0   
47   NaN   NaN   NaN   NaN  NaN  17.0   6.0           14.0   
48   NaN   NaN   NaN   NaN  NaN  17.0   5.0           14.0   
49   NaN   NaN   9.0   NaN  5.0   6.0   3.0           16.0   
50   NaN   NaN   9.0   NaN  5.0   6.0   2.0           17.0   
51   NaN   NaN   9.0   NaN  5.0   6.0   1.0           17.0   
52   NaN   NaN   1.0  12.0  3.0   3.0   1.0           18.0   
53  25.0   NaN   NaN   NaN  NaN   NaN   1.0           20.0   
54   NaN   NaN   NaN   NaN  NaN  17.0   4.0           20.0   
55   NaN   NaN   NaN  19.0  NaN   NaN   1.0           20.0   
56   NaN   NaN   NaN   NaN  NaN  17.0   3.0           23.0   
57   NaN   NaN   NaN   NaN  NaN  17.0   2.0           23.0   
58   NaN   NaN   NaN   NaN  NaN  17.0   1.0           23.0   

    Total LM Order QTY Width Scheduled Ship Date  Order Number  
0         2.941176e+06   170          2020-08-05             1  
1         2.941176e+06   170          2020-08-05             1  
2         2.941176e+06   170          2020-08-06             2  
3         2.941176e+06  

In [91]:
setup_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx', sheet_name='Deckle Parameters')
speed_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx', sheet_name='Product Parameters')

In [92]:
setup_df

Slitter Set-up               Time (minutes)  \
0                            AMB & Arium                           50   
1                          Normal Knife                            15   
2                    Knife and Manderel                            30   
3                           <100mm slits                           30   
4                      Jumbo Roll Set-up               Time (minutes)   
5                        Jumbo roll only                           15   
6                   Jumbo roll and knife                           30   
7      Jumbo roll and knife and manderal                           30   
8            Jumbo roll and <100mm slits                           45   
9               Slit Off-set definitiaon  Target width + Slit Off-set   
10                    Slit Combinations              # of slit widths   
11    # of slit combinations - preferred                            3   
12               Max # slit combinations                            4   
13  Abiltiy to run 2,3 or 4 combinations        (wrapper bottle-neck)   
14              Slit Off-set definitiaon  Target width + Slit Off-set   

                                           Unnamed: 2  
0   arium is a product. Sounds like this happens b...  
1                                                 NaN  
2                                                 NaN  
3                                                 NaN  
4                                                 NaN  
5                                                 NaN  
6                                                 NaN  
7                                                 NaN  
8                                                 NaN  
9                                                 NaN  
10                                                NaN  
11                                                NaN  
12                                                NaN  
13                                                NaN  
14                                                NaN

In [93]:
slitter_speed = speed_df.loc[(speed_df['Customer Name'] == customer) &
             (speed_df['Description'].str.contains(technology)) &
             (speed_df['Description'].str.contains(color)), 'Slittter Speed (m/min)'].reset_index(drop=True)[0]
slitter_speed

1300

In [94]:
# add column 'completion date/time'
# for row in schedule, calculate the completion time
sorted_schedule_with_order_info['Completion Date'] = None

### choose starte date/time
start_date_time = datetime.datetime(2020, 8, 12)
print("start time: {}".format(start_date_time))

### set changeover times
jumbo_change = setup_df.loc[setup_df['Slitter Set-up'] == 
                            'Jumbo roll only']['Time (minutes)'].values[0]
jumbo_change = datetime.timedelta(minutes=jumbo_change)
print("jumbo change: {}".format(jumbo_change))
jumbo_and_knife_change = setup_df.loc[setup_df['Slitter Set-up'] == 
                                      'AMB & Arium']['Time (minutes)'].values[0]
jumbo_and_knife_change = datetime.timedelta(minutes=jumbo_and_knife_change)
print("jumbo and knife change: {}".format(jumbo_and_knife_change))

### print slitter speed and doff length
print("slitter speed (m/min): {}".format(slitter_speed))
print("doff length (m): {}".format(doff_length))
prior_completion_date_time = start_date_time
match = '\d\d\d'
layout_columns = [i for i in sorted_schedule_with_order_info.columns if re.match(match, i)]

### zero-out the 'prior layout' so algorithm knows jumbo + knife change
prior_layout = sorted_schedule_with_order_info.iloc[0][layout_columns]
for col in prior_layout.index:
    prior_layout[col] = 0
layout_number = 0

schedule_with_change_over = pd.DataFrame()
### transition times are for loading the current jumbo
for row in sorted_schedule_with_order_info.index:
    
    layout = sorted_schedule_with_order_info.iloc[row][layout_columns]
    if all(prior_layout.fillna(0) == layout.fillna(0)):
        transition_time = jumbo_change
    else:
        transition_time = jumbo_and_knife_change
        layout_number += 1
    print("transition time: {}".format(transition_time))
    run_time = doff_length / slitter_speed
    run_time = datetime.timedelta(minutes=run_time)
    print("run time: {}".format(run_time))
    completion_time = transition_time + run_time
    completion_date_time = prior_completion_date_time + completion_time
    print("completion date/time: {}".format(completion_date_time))
    
    ### change main df
    sorted_schedule_with_order_info['Completion Date'][row] = completion_date_time
    sorted_schedule_with_order_info['layout number'][row] = layout_number
    
    ### with changeover rows
    current_changeover = pd.DataFrame(sorted_schedule_with_order_info.columns)
    current_changeover = current_changeover.set_index(0)
    current_changeover = current_changeover.T
    current_changeover = current_changeover.append({'Order Number': 'Changeover'}, ignore_index=True)
    current_changeover['Completion Date'] = prior_completion_date_time + transition_time
    current_order = pd.DataFrame(sorted_schedule_with_order_info.iloc[row]).T
    current_combined = current_changeover.append(current_order, sort=False, ignore_index=True)
    
    schedule_with_change_over = pd.concat([schedule_with_change_over, current_combined])
    
    prior_completion_date_time = completion_date_time
    prior_layout = layout
    clear_output(wait=True)
    
schedule_with_change_over = schedule_with_change_over.reset_index(drop=True)

transition time: 0:15:00
run time: 1:18:27.692308
completion date/time: 2020-08-16 05:14:13.846172


In [95]:
schedule_with_change_over = schedule_with_change_over.reset_index(drop=True)

In [96]:
schedule_with_change_over.head(40)

0   154  158  170  208  218  234 freq layout number Total LM Order QTY Width  \
0   NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
1   NaN  NaN   23  NaN  NaN  NaN    1             1        2.94118e+06   170   
2   NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
3   NaN  NaN   15    7  NaN  NaN    1             2        2.94118e+06   170   
4   NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
5   NaN  NaN    9  NaN    5    6   31             3        2.94118e+06   170   
6   NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
7   NaN  NaN    9  NaN    5    6   30             3        2.94118e+06   170   
8   NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
9   NaN  NaN    9  NaN    5    6   29             3        2.94118e+06   170   
10  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
11   26  NaN  NaN  NaN  NaN  NaN    2             4        3.24675e+06   154   
12  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
13   26  NaN  NaN  NaN  NaN  NaN    1             4        3.24675e+06   154   
14  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
15  NaN  NaN    9  NaN    5    6   28             5        2.94118e+06   170   
16  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
17  NaN  NaN    9  NaN    5    6   27             5        2.94118e+06   170   
18  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
19  NaN  NaN    9  NaN    5    6   26             5        2.94118e+06   170   
20  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
21  NaN  NaN    9  NaN    5    6   25             5        2.94118e+06   170   
22  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
23  NaN  NaN    9  NaN    5    6   24             5        2.94118e+06   170   
24  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
25  NaN  NaN    9  NaN    5    6   23             5        2.94118e+06   170   
26  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
27  NaN  NaN    9  NaN    5    6   22             5        2.29358e+06   218   
28  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
29  NaN  NaN    9  NaN    5    6   21             5        2.29358e+06   218   
30  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
31  NaN  NaN    9  NaN    5    6   20             5        2.29358e+06   218   
32  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
33  NaN  NaN    9  NaN    5    6   19             5        2.29358e+06   218   
34  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
35  NaN  NaN    9  NaN    5    6   18             5        2.29358e+06   218   
36  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
37  NaN   25  NaN  NaN  NaN  NaN    1             6        3.16456e+06   158   
38  NaN  NaN  NaN  NaN  NaN  NaN  NaN           NaN                NaN   NaN   
39   19    7  NaN  NaN  NaN  NaN    1             7        3.16456e+06   158   

0  Scheduled Ship Date Order Number            Completion Date  
0                  NaT   Changeover 2020-08-12 00:50:00.000000  
1           2020-08-05            1 2020-08-12 02:08:27.692308  
2                  NaT   Changeover 2020-08-12 02:58:27.692308  
3           2020-08-05            1 2020-08-12 04:16:55.384616  
4                  NaT   Changeover 2020-08-12 05:06:55.384616  
5           2020-08-06            2 2020-08-12 06:25:23.076924  
6                  NaT   Changeover 2020-08-12 06:40:23.076924  
7           2020-08-06            2 2020-08-12 07:58:50.769232  
8                  NaT   Changeover 2020-08-12 08:13:50.769232  
9           2020-08-06            2 2020-08-12 09:32:18.461540  
10  

In [97]:
# summary w changeover rows

master_schedule = pd.DataFrame()
unformatted_schedule = pd.DataFrame()
for index in schedule_with_change_over.index[1::2]: # pass through every jumbo
    order_number = schedule_with_change_over.iloc[index]['Order Number']
    
    ### these are columns specific to width layouts
    match = '\d\d\d'
    layout_columns = [i for i in schedule_with_change_over.columns if re.match(match, i)]
    
    
    deckle_layout =  deckle_order = pd.DataFrame(schedule_with_change_over.iloc[index]).T
    
    ### these are columns specific to order details
    order_columns = [i for i in deckle_order.columns if (i not in layout_columns)
                     & (i not in ['freq', 'layout number'])]
    
    ### for order we grab the last row since this is the real 'completion date'
    order = deckle_order[order_columns].reset_index(drop=True).iloc[-1].dropna()
    
    ### remove widths that are in the deckle opt. but not in the layout
    formula = deckle_layout[layout_columns].reset_index(drop=True).iloc[0].dropna()
    
    ### number of doffs for the given layout/order
    doffs = doffs_in_jumbo
    read_out = ''
    for i in range(formula.shape[0]-1): #for unique prods
        read_out = read_out + ("{}x{} + ".format(formula.index[i], 
                                                formula.iloc[i].astype(int)))
    read_out = read_out + ("{}x{}".format(formula.index[-1], 
                                        formula.iloc[-1].astype(int))) #make last string w/o +
    current = pd.DataFrame([read_out, int(doffs), order]).T
    order = pd.DataFrame(order).T
    order = order.reset_index(drop=True)
    current = pd.DataFrame([read_out, int(doffs)]).T
    current.columns = ['Formula', 'Doffs']
    current = current.join(order)
    
    ### add changeover info
    current_changeover = pd.DataFrame(current.columns)
    current_changeover = current_changeover.set_index(0)
    current_changeover = current_changeover.T
    current_changeover = current_changeover.append({'Order Number': 'Changeover'}, ignore_index=True)
    current_changeover['Completion Date'] = schedule_with_change_over.iloc[index-1]['Completion Date']
    current_combined = current_changeover.append(current, sort=False, ignore_index=True)
    current_combined
#     temp = pd.DataFrame(formula).T
#     temp2 = pd.DataFrame(order).T
#     unformatted_schedule = pd.concat([unformatted_schedule, temp, temp2], sort=False)
#     current.columns = ['Formula', 'Doffs']
    master_schedule = pd.concat([master_schedule, current_combined])
    master_schedule = master_schedule.reset_index(drop=True)

In [98]:
master_schedule.head(50)

0                 Formula Doffs Total LM Order QTY Width Scheduled Ship Date  \
0                     NaN   NaN                NaN   NaN                 NaT   
1                  170x23     6        2.94118e+06   170          2020-08-05   
2                     NaN   NaN                NaN   NaN                 NaT   
3          170x15 + 208x7     6        2.94118e+06   170          2020-08-05   
4                     NaN   NaN                NaN   NaN                 NaT   
5   170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-06   
6                     NaN   NaN                NaN   NaN                 NaT   
7   170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-06   
8                     NaN   NaN                NaN   NaN                 NaT   
9   170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-06   
10                    NaN   NaN                NaN   NaN                 NaT   
11                 154x26     6        3.24675e+06   154          2020-08-07   
12                    NaN   NaN                NaN   NaN                 NaT   
13                 154x26     6        3.24675e+06   154          2020-08-07   
14                    NaN   NaN                NaN   NaN                 NaT   
15  170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-07   
16                    NaN   NaN                NaN   NaN                 NaT   
17  170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-07   
18                    NaN   NaN                NaN   NaN                 NaT   
19  170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-07   
20                    NaN   NaN                NaN   NaN                 NaT   
21  170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-07   
22                    NaN   NaN                NaN   NaN                 NaT   
23  170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-07   
24                    NaN   NaN                NaN   NaN                 NaT   
25  170x9 + 218x5 + 234x6     6        2.94118e+06   170          2020-08-07   
26                    NaN   NaN                NaN   NaN                 NaT   
27  170x9 + 218x5 + 234x6     6        2.29358e+06   218          2020-08-07   
28                    NaN   NaN                NaN   NaN                 NaT   
29  170x9 + 218x5 + 234x6     6        2.29358e+06   218          2020-08-07   
30                    NaN   NaN                NaN   NaN                 NaT   
31  170x9 + 218x5 + 234x6     6        2.29358e+06   218          2020-08-07   
32                    NaN   NaN                NaN   NaN                 NaT   
33  170x9 + 218x5 + 234x6     6        2.29358e+06   218          2020-08-07   
34                    NaN   NaN                NaN   NaN                 NaT   
35  170x9 + 218x5 + 234x6     6        2.29358e+06   218          2020-08-07   
36                    NaN   NaN                NaN   NaN                 NaT   
37                 158x25     6        3.16456e+06   158          2020-08-13   
38                    NaN   NaN                NaN   NaN                 NaT   
39         154x19 + 158x7     6        3.16456e+06   158          2020-08-13   
40                    NaN   NaN                NaN   NaN                 NaT   
41                 208x19     6        2.40385e+06   208          2020-08-14   
42                    NaN   NaN                NaN   NaN                 NaT   
43                 208x19     6        2.40385e+06   208          2020-08-14   
44                    NaN   NaN                NaN   NaN                 NaT   
45                 208x19     6        2.40385e+06   208          2020-08-14   
46                    NaN   NaN                NaN   NaN                 NaT   
47  170x9 + 218x5 + 234x6     6        2.29358e+06   218          2020-08-14   
48                    NaN   NaN                NaN   NaN                 NaT   


In [99]:
# master_schedule.to_excel('layouts with completion dates and changeover.xlsx')

In [100]:
# now condense
# do we want to groupby orders, layouts, both?
dff = pd.DataFrame(sorted_schedule_with_order_info.\
                   groupby(['layout number', 'Order Number']).size()).\
                   rename(columns={0: 'Doffs'}).reset_index()

In [101]:
# summary w/o changeover rows

master_schedule = pd.DataFrame()
unformatted_schedule = pd.DataFrame()
for index in dff.index: # pass through layout/order combinations
    order_number = dff.iloc[index]['Order Number']
    
    ### these are columns specific to width layouts
    match = '\d\d\d'
    layout_columns = [i for i in sorted_schedule_with_order_info.columns if re.match(match, i)]
    
    ### match dff grouping with order/layout info in original schedule
    deckle_layout = sorted_schedule_with_order_info.loc\
                    [sorted_schedule_with_order_info['layout number']\
                     == dff.iloc[index]['layout number']]
    
    ### since layout/orders are pareto-ed out in schedule, need to match both
    ### for order completion
    deckle_order =  sorted_schedule_with_order_info.loc\
                    [(sorted_schedule_with_order_info['Order Number']\
                     == dff.iloc[index]['Order Number']) &
                     (sorted_schedule_with_order_info['layout number']\
                     == dff.iloc[index]['layout number'])]
    
    ### these are columns specific to order details
    order_columns = [i for i in deckle_order.columns if (i not in layout_columns)
                     & (i not in ['freq', 'layout number'])]
    
    ### for order we grab the last row since this is the real 'completion date'
    order = deckle_order[order_columns].reset_index(drop=True).iloc[-1].dropna()
    
    ### remove widths that are in the deckle opt. but not in the layout
    formula = deckle_layout[layout_columns].reset_index(drop=True).iloc[0].dropna()
    
    ### number of doffs for the given layout/order
    doffs = dff.iloc[index]['Doffs']
    read_out = ''
    for i in range(formula.shape[0]-1): #for unique prods
        read_out = read_out + ("{}x{} + ".format(formula.index[i], 
                                                formula.iloc[i].astype(int)))
    read_out = read_out + ("{}x{}".format(formula.index[-1], 
                                        formula.iloc[-1].astype(int))) #make last string w/o +
    current = pd.DataFrame([read_out, int(doffs), order]).T
    order = pd.DataFrame(order).T
    order = order.reset_index(drop=True)
    current = pd.DataFrame([read_out, int(doffs)]).T
    current.columns = ['Formula', 'Doffs']
    current = current.join(order)
#     temp = pd.DataFrame(formula).T
#     temp2 = pd.DataFrame(order).T
#     unformatted_schedule = pd.concat([unformatted_schedule, temp, temp2], sort=False)
#     current.columns = ['Formula', 'Doffs']
    master_schedule = pd.concat([master_schedule, current])
    master_schedule = master_schedule.reset_index(drop=True)

In [102]:
master_schedule['Doffs'] = master_schedule['Doffs']*doffs_in_jumbo

In [103]:
# master_schedule.to_excel('layouts with completion dates.xlsx')

In [104]:
master_schedule

Formula Doffs Total LM Order QTY Width  \
0                           170x23     6        2.94118e+06   170   
1                   170x15 + 208x7     6        2.94118e+06   170   
2            170x9 + 218x5 + 234x6    18        2.94118e+06   170   
3                           154x26    12        3.24675e+06   154   
4            170x9 + 218x5 + 234x6    18        2.94118e+06   170   
5            170x9 + 218x5 + 234x6    18        2.94118e+06   170   
6            170x9 + 218x5 + 234x6     6        2.29358e+06   218   
7            170x9 + 218x5 + 234x6    24        2.29358e+06   218   
8                           158x25     6        3.16456e+06   158   
9                   154x19 + 158x7     6        3.16456e+06   158   
10                          208x19     6        2.40385e+06   208   
11                          208x19    12        2.40385e+06   208   
12           170x9 + 218x5 + 234x6    30        2.29358e+06   218   
13           170x9 + 218x5 + 234x6    24        2.29358e+06   218   
14           170x9 + 218x5 + 234x6    30        2.29358e+06   218   
15                          234x17     6        2.13675e+06   234   
16                          234x17     6        2.13675e+06   234   
17                          234x17     6        2.13675e+06   234   
18                          234x17     6        2.13675e+06   234   
19                          234x17     6        2.13675e+06   234   
20                          234x17    12        2.13675e+06   234   
21                          234x17     6        2.13675e+06   234   
22                          234x17     6        2.13675e+06   234   
23                          234x17     6        2.13675e+06   234   
24                          234x17    12        2.13675e+06   234   
25           170x9 + 218x5 + 234x6     6        2.94118e+06   170   
26           170x9 + 218x5 + 234x6    12        2.29358e+06   218   
27  170x1 + 208x12 + 218x3 + 234x3     6        2.29358e+06   218   
28                          154x25     6        3.24675e+06   154   
29                          234x17     6        2.13675e+06   234   
30                          208x19     6        2.40385e+06   208   
31                          234x17     6        2.13675e+06   234   
32                          234x17     6        2.13675e+06   234   
33                          234x17     6        2.13675e+06   234   

   Scheduled Ship Date Order Number            Completion Date  
0           2020-08-05            1 2020-08-12 02:08:27.692308  
1           2020-08-05            1 2020-08-12 04:16:55.384616  
2           2020-08-06            2 2020-08-12 09:32:18.461540  
3           2020-08-07            3 2020-08-12 13:14:13.846156  
4           2020-08-07            4 2020-08-12 18:29:36.923080  
5           2020-08-07            5 2020-08-12 23:10:00.000004  
6           2020-08-07            7 2020-08-13 00:43:27.692312  
7           2020-08-07            8 2020-08-13 06:57:18.461544  
8           2020-08-13           10 2020-08-13 09:05:46.153852  
9           2020-08-13           10 2020-08-13 11:14:13.846160  
10          2020-08-14           12 2020-08-13 13:22:41.538468  
11          2020-08-14           13 2020-08-13 16:29:36.923084  
12          2020-08-14           14 2020-08-14 00:51:55.384624  
13          2020-08-17           18 2020-08-14 07:05:46.153856  
14          2020-08-19           23 2020-08-14 14:53:04.615396  
15          2020-08-21           26 2020-08-14 17:01:32.307704  
16          2020-08-21           27 2020-08-14 18:35:00.000012  
17          2020-08-21           28 2020-08-14 20:08:27.692320  
18          2020-08-21           29 2020-08-14 21:41:55.384628  
19          2020-08-24           30 2020-08-14 23:15:23.076936  
20          2020-08-24           31 2020-08-15 02:22:18.461552  
21          2020-08-24           32 2020-08-15 03:55:46.153860  
22          2020-08-24           33 2020-08-15 05:29:13.846168  
23          2020-08-24           34 2020-08-15 07:0

In [105]:
df['Date order is complete']

0     2020-07-20 13:05:05.263
1     2020-07-20 16:00:31.579
2     2020-07-20 19:34:33.684
3     2020-07-20 23:22:37.895
4     2020-07-21 03:10:42.105
                ...          
355   2020-12-25 14:58:21.064
356   2020-12-25 15:28:21.064
357   2020-12-27 20:40:17.958
358   2020-12-28 11:02:59.281
359   2020-12-28 23:02:59.281
Name: Date order is complete, Length: 360, dtype: datetime64[ns]

In [106]:
def optimize_late_orders(sol, widths, neckin, df, L, set_readable=True):
    """
    Parameters
    ----------
    sol: list
        optimized layouts
    widths: list
        list of product widths
    neckin: list
        list of product neckins
    df: DataFrame
        pandas DataFrame of schedule
    L: int
        put up
    set_readable: Boolean, True False
        if True will make the schedule in Berry format

    """
    extras = pd.DataFrame(np.zeros(len(widths))).T
    master2 = make_layout_registrar(sol, widths, neckin)
    extras.columns = master2.columns[:-1]
    schedule = []
    layout_pattern = 0
    old_width = width = None
    for row1 in df.index:
        current = df.iloc[row1][['Total LM Order QTY', 'Width', 'Scheduled Ship Date']]
        doffs = math.ceil(current['Total LM Order QTY'] / L) # QTY
        width = current['Width']
        if width == old_width:
            layout_pattern -= 1
        master2 = master2.sort_values(width, ascending=False).reset_index(drop=True)
        target_doffs = extras.iloc[0][width]
        print(doffs, width, target_doffs)
        for row in master2.index:
            layout_pattern += 1
            for count in range(master2.iloc[row]['freq'].astype(int)):
                target_doffs += master2.iloc[row][width]
                master2.at[row, 'layout number'] = layout_pattern
                schedule.append(master2.iloc[row])
                master2.at[row, 'freq'] = master2.at[row, 'freq'] - 1

                for master_index in master2.iloc[row].index[:-2]:
                    if (math.isnan(master2.iloc[row][master_index]) == False):
                        extras.iloc[0][master_index] = extras.iloc[0][master_index] + master2.iloc[row][master_index]
                if target_doffs > doffs:
                    break
            else:
                continue
            break
        old_width = width
        extra = target_doffs - doffs
        extras.iloc[0][width] = extra
        clear_output(wait=True)
        print(extras)
        
    
        master_schedule = pd.DataFrame()
        unformatted_schedule = pd.DataFrame()
        sorted_schedule = pd.DataFrame(schedule)
        dff = pd.DataFrame(sorted_schedule.groupby('layout number').size()).rename(columns={0: 'Doffs'}).reset_index()
        for layout_number in dff['layout number']:
            deckle = sorted_schedule.loc[sorted_schedule['layout number'] == layout_number]
            formula = deckle.reset_index(drop=True).iloc[0].dropna()
            freq = dff.loc[dff['layout number'] == layout_number]['Doffs'].values[0]
            read_out = ''
            for i in range(formula.shape[0]-3): #for unique prods
                read_out = read_out + ("{}x{} + ".format(formula.index[i], formula.iloc[i].astype(int)))

            read_out = read_out + ("{}x{}".format(formula.index[-3], formula.iloc[-3].astype(int))) #make last string w/o +
            current = pd.DataFrame([read_out, int(freq)]).T
            temp = pd.DataFrame(formula).T
            temp = temp[temp.columns[:-1]]
            unformatted_schedule = pd.concat([unformatted_schedule, temp], sort=False)
            current.columns = ['Formula', 'Doffs']
            master_schedule = pd.concat([master_schedule, current])
        master_schedule = master_schedule.reset_index(drop=True)
    if set_readable == False:
        master_schedule = unformatted_schedule
        
    return master_schedule

In [107]:
# master_schedule.to_excel('temp.xlsx', index=False)

In [109]:
for i in sol:
    for j in i:
        if j > 0:
            print(j)

225.0
175.0
175.0
241.0
175.0
175.0
225.0
175.0
241.0
175.0
241.0
175.0
225.0
241.0
175.0
225.0
241.0
175.0
241.0
225.0
225.0
175.0
175.0
241.0
175.0
175.0
225.0
175.0
241.0
175.0
241.0
175.0
225.0
241.0
175.0
225.0
241.0
175.0
241.0
225.0
225.0
175.0
175.0
241.0
175.0
175.0
225.0
175.0
241.0
175.0
241.0
175.0
225.0
241.0
175.0
225.0
241.0
175.0
241.0
225.0
225.0
175.0
175.0
241.0
175.0
175.0
225.0
175.0
241.0
175.0
241.0
175.0
225.0
241.0
175.0
225.0
241.0
175.0
241.0
225.0
225.0
175.0
175.0
241.0
175.0
175.0
225.0
175.0
241.0
175.0
241.0
175.0
225.0
241.0
175.0
225.0
241.0
175.0
241.0
225.0
225.0
175.0
175.0
241.0
175.0
175.0
225.0
175.0
241.0
175.0
241.0
175.0
225.0
241.0
175.0
225.0
241.0
175.0
241.0
225.0
225.0
175.0
175.0
241.0
175.0
175.0
225.0
175.0
241.0
175.0
241.0
175.0
225.0
241.0
175.0
225.0
241.0
175.0
241.0
225.0
225.0
175.0
175.0
241.0
175.0
175.0
225.0
175.0
241.0
175.0
241.0
175.0
225.0
241.0
175.0
225.0
241.0
175.0
241.0
225.0
225.0
175.0
175.0
241.0
175.0
175.0
225.

In [115]:
start = str(df_filtered['Date order is complete'][0])

In [123]:
start.split('.')[0]

'2020-08-12 18:22:30'

In [124]:
datetime.datetime.strptime(start.split('.')[0], '%Y-%m-%d %H:%M:%S')

datetime.datetime(2020, 8, 12, 18, 22, 30)